# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
# Config to make development easier 

# turns off Jedi to solve slow tab auto completeion https://github.com/ipython/ipython/issues/10493
%config Completer.use_jedi = False 

In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [3]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [4]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [5]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [6]:
encoded[:100]

array([17, 64, 52, 77, 43, 20, 57,  2,  5, 46, 46, 46, 30, 52, 77, 77, 22,
        2, 39, 52, 70, 80,  3, 80, 20,  0,  2, 52, 57, 20,  2, 52,  3,  3,
        2, 52,  3, 80, 14, 20, 29,  2, 20, 32, 20, 57, 22,  2, 45, 56, 64,
       52, 77, 77, 22,  2, 39, 52, 70, 80,  3, 22,  2, 80,  0,  2, 45, 56,
       64, 52, 77, 77, 22,  2, 80, 56,  2, 80, 43,  0,  2, 44, 82, 56, 46,
       82, 52, 22, 23, 46, 46, 38, 32, 20, 57, 22, 43, 64, 80, 56])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [7]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [9]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    n_batches = len(arr)//(batch_size*seq_length)
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:(n_batches*batch_size*seq_length)]
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [10]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [11]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[17 64 52 77 43 20 57  2  5 46]
 [ 0 44 56  2 43 64 52 43  2 52]
 [20 56 18  2 44 57  2 52  2 39]
 [ 0  2 43 64 20  2 72 64 80 20]
 [ 2  0 52 82  2 64 20 57  2 43]
 [72 45  0  0 80 44 56  2 52 56]
 [ 2 71 56 56 52  2 64 52 18  2]
 [41 79  3 44 56  0 14 22 23  2]]

y
 [[64 52 77 43 20 57  2  5 46 46]
 [44 56  2 43 64 52 43  2 52 43]
 [56 18  2 44 57  2 52  2 39 44]
 [ 2 43 64 20  2 72 64 80 20 39]
 [ 0 52 82  2 64 20 57  2 43 20]
 [45  0  0 80 44 56  2 52 56 18]
 [71 56 56 52  2 64 52 18  2  0]
 [79  3 44 56  0 14 22 23  2 73]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [12]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [23]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(len(self.chars), self.n_hidden, self.n_layers, dropout=self.drop_prob, batch_first=True)
        self.dropout = nn.Dropout(self.drop_prob)
        self.fc1 = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        
        # reccurent layer(s)
        output, hidden = self.lstm(x, hidden)
        
        # Dropout 
        output = self.dropout(output)
        
        # reshaping 
        output = output.contiguous().view(-1, self.n_hidden)
        
        # Fully connected 
        out = self.fc1(output)
        
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [24]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [27]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=512, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [29]:
batch_size = 128
seq_length = 100
n_epochs =  30 # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/30... Step: 10... Loss: 1.6367... Val Loss: 1.5969
Epoch: 1/30... Step: 20... Loss: 1.6331... Val Loss: 1.5797
Epoch: 1/30... Step: 30... Loss: 1.6082... Val Loss: 1.5702
Epoch: 1/30... Step: 40... Loss: 1.5726... Val Loss: 1.5676
Epoch: 1/30... Step: 50... Loss: 1.6089... Val Loss: 1.5661
Epoch: 1/30... Step: 60... Loss: 1.5317... Val Loss: 1.5616
Epoch: 1/30... Step: 70... Loss: 1.5517... Val Loss: 1.5579
Epoch: 1/30... Step: 80... Loss: 1.5406... Val Loss: 1.5542
Epoch: 1/30... Step: 90... Loss: 1.5563... Val Loss: 1.5469
Epoch: 1/30... Step: 100... Loss: 1.5410... Val Loss: 1.5443
Epoch: 1/30... Step: 110... Loss: 1.5354... Val Loss: 1.5432
Epoch: 1/30... Step: 120... Loss: 1.5141... Val Loss: 1.5348
Epoch: 1/30... Step: 130... Loss: 1.5653... Val Loss: 1.5335
Epoch: 2/30... Step: 140... Loss: 1.5903... Val Loss: 1.5288
Epoch: 2/30... Step: 150... Loss: 1.5468... Val Loss: 1.5282
Epoch: 2/30... Step: 160... Loss: 1.5526... Val Loss: 1.5184
Epoch: 2/30... Step: 170... Loss:

Epoch: 10/30... Step: 1350... Loss: 1.2404... Val Loss: 1.3213
Epoch: 10/30... Step: 1360... Loss: 1.2491... Val Loss: 1.3217
Epoch: 10/30... Step: 1370... Loss: 1.2404... Val Loss: 1.3184
Epoch: 10/30... Step: 1380... Loss: 1.2836... Val Loss: 1.3181
Epoch: 10/30... Step: 1390... Loss: 1.3001... Val Loss: 1.3165
Epoch: 11/30... Step: 1400... Loss: 1.2820... Val Loss: 1.3214
Epoch: 11/30... Step: 1410... Loss: 1.2989... Val Loss: 1.3149
Epoch: 11/30... Step: 1420... Loss: 1.2987... Val Loss: 1.3116
Epoch: 11/30... Step: 1430... Loss: 1.2619... Val Loss: 1.3149
Epoch: 11/30... Step: 1440... Loss: 1.2816... Val Loss: 1.3201
Epoch: 11/30... Step: 1450... Loss: 1.2168... Val Loss: 1.3157
Epoch: 11/30... Step: 1460... Loss: 1.2499... Val Loss: 1.3139
Epoch: 11/30... Step: 1470... Loss: 1.2320... Val Loss: 1.3157
Epoch: 11/30... Step: 1480... Loss: 1.2492... Val Loss: 1.3109
Epoch: 11/30... Step: 1490... Loss: 1.2452... Val Loss: 1.3099
Epoch: 11/30... Step: 1500... Loss: 1.2372... Val Loss:

Epoch: 20/30... Step: 2660... Loss: 1.1625... Val Loss: 1.2569
Epoch: 20/30... Step: 2670... Loss: 1.1741... Val Loss: 1.2610
Epoch: 20/30... Step: 2680... Loss: 1.1685... Val Loss: 1.2533
Epoch: 20/30... Step: 2690... Loss: 1.1566... Val Loss: 1.2558
Epoch: 20/30... Step: 2700... Loss: 1.1661... Val Loss: 1.2568
Epoch: 20/30... Step: 2710... Loss: 1.1340... Val Loss: 1.2559
Epoch: 20/30... Step: 2720... Loss: 1.1316... Val Loss: 1.2601
Epoch: 20/30... Step: 2730... Loss: 1.1271... Val Loss: 1.2543
Epoch: 20/30... Step: 2740... Loss: 1.1325... Val Loss: 1.2525
Epoch: 20/30... Step: 2750... Loss: 1.1325... Val Loss: 1.2517
Epoch: 20/30... Step: 2760... Loss: 1.1334... Val Loss: 1.2593
Epoch: 20/30... Step: 2770... Loss: 1.1685... Val Loss: 1.2566
Epoch: 20/30... Step: 2780... Loss: 1.1921... Val Loss: 1.2587
Epoch: 21/30... Step: 2790... Loss: 1.1651... Val Loss: 1.2569
Epoch: 21/30... Step: 2800... Loss: 1.1900... Val Loss: 1.2522
Epoch: 21/30... Step: 2810... Loss: 1.1863... Val Loss:

Epoch: 29/30... Step: 3970... Loss: 1.1244... Val Loss: 1.2407
Epoch: 29/30... Step: 3980... Loss: 1.0874... Val Loss: 1.2481
Epoch: 29/30... Step: 3990... Loss: 1.0933... Val Loss: 1.2442
Epoch: 29/30... Step: 4000... Loss: 1.0895... Val Loss: 1.2432
Epoch: 29/30... Step: 4010... Loss: 1.0876... Val Loss: 1.2452
Epoch: 29/30... Step: 4020... Loss: 1.0903... Val Loss: 1.2403
Epoch: 29/30... Step: 4030... Loss: 1.0991... Val Loss: 1.2470
Epoch: 30/30... Step: 4040... Loss: 1.0982... Val Loss: 1.2416
Epoch: 30/30... Step: 4050... Loss: 1.0978... Val Loss: 1.2488
Epoch: 30/30... Step: 4060... Loss: 1.1117... Val Loss: 1.2482
Epoch: 30/30... Step: 4070... Loss: 1.1106... Val Loss: 1.2418
Epoch: 30/30... Step: 4080... Loss: 1.0966... Val Loss: 1.2368
Epoch: 30/30... Step: 4090... Loss: 1.0997... Val Loss: 1.2395
Epoch: 30/30... Step: 4100... Loss: 1.0815... Val Loss: 1.2452
Epoch: 30/30... Step: 4110... Loss: 1.0772... Val Loss: 1.2405
Epoch: 30/30... Step: 4120... Loss: 1.0789... Val Loss:

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [30]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [31]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [32]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [33]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna Alexandrovna, a step old prince
with his brother, as sterely began the counting her son with a
party with her and sighed and walked to him, and she caught the sound of his
cheeks, and the cracked forest was a good-but calting compasts of the marsh
from his companion to the sigrificance, he was thronging on all his head and
fresh stress of her husband, he was tasting to hild behind the parasol, he felt that
that she was not in a sense of articulity in his eyes, and that the sick of
the country could say nothing; but the children had been a spring and an amperted
tables, hinder to she himself with his wife. But his brother was a
silence when he had to be silent and without home to her.

"What do you think about it. There's no right to be so that I am,
to be a child."

"You know, that's taking up here," said Varenka and her husband's foot
of the coat, sighing with smile, "they're to be stepped on a capital
fordic means to tast it as to how, the case we have a man, shall not
be influe

In [42]:
print(sample(net, 1000, prime='Matilda name day', top_k=5))

Matilda name day
as he's former hopelessness, and his clishes sounds to his brother trying to
speak for that candle, who had been the point of thinking of the career with
her, he could. His brother showed them all over the past.



Chapter 29


When the priest was an inclination was to dress with her, and that his
work with her son, when the marshal was coming to this side of
the cold and manner. The presence of that manner of the portrait of this mercely
would have and seeing him tea, and the center of that company was a passion,
and and was difficult to be angry, would be suddenly contemptuous to
all another province; the princess's
person should see his brother, and that they came to see her the past that
he had no time to decoin it all her husband's and went off,
his head, and so that she could not help believed that he did not
believe his first passion for his back to himself the peasant's accompasiment.

"What was it is alr abroad, I was thinking me, I see it out on the
country."

## Loading a checkpoint

In [34]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [35]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said nothing. He had
seen him as to be discided, and she was seening his brother, and
shook himself. And all that was the minutest that had a portrait of the most cruel,
a frown of the servants and the same acquaintance of the study to admit the
labor housed there would have long been many another, but she
had already breaked over the chair at his words.

Their conversating was if the haring he sat stringing a seel of special
suffering and all the stupin soul to her. His wife suddenly said in
his hostility. All alone, to call into the miller in his eyes. He stopped him from the
countess.

"And what do you say, I won't be above throothing. I don't know."

"I'm solry words for your conviction of those thoughts about him; I've not
told you a bit. There will show than such is for the first time to
go away, seeing her that I'm going with him. This wanted to go to him," anstered Levin,
still show himself that he was not like a son, and asking her to be a
misery. His breast had not 